In [1]:
import time

import warnings
warnings.filterwarnings('ignore')
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [3]:
from selenium.common.exceptions import NoSuchElementException

driver_path = 'C:/Users/임채현/[숙탯]겨울/chromedriver.exe'

# ChromeOptions 설정
chrome_options = webdriver.ChromeOptions()


# 다른 옵션들을 추가하려면 여기에 추가
#service = Service(executable_path=driver_path)

# 크롬 드라이버 초기화
driver = webdriver.Chrome( options=chrome_options)
driver.get('https://map.naver.com/v5/search')

# 팝업 창 제거
#driver.find_element_by_css_selector("button#intro_popup_close").click()


time.sleep(5)


#애월맛집 검색, 맛집 리스트 뽑기
search_input = driver.find_element(By.CSS_SELECTOR, 'div.input_box input.input_search')
search_input.click()
search_input.send_keys("애월맛집")
time.sleep(3)

search_input.send_keys(Keys.ENTER)
time.sleep(5)

#프레임 변경
frame_xpath = '//*[@id="searchIframe"]'
frame = WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, frame_xpath))
)

#필터링 - 리뷰 많은 순으로 설정
##필터링 버튼 클릭
fil_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div[1]/div/div/div/div/div/span[1]/a'))
    )
fil_button.click()
time.sleep(5)

##리뷰많은순 클릭
element_css_selector = "#_place_portal_root > div > div.B1gsM > div.YWnDg > div > div > div:nth-child(1) > div.RhjQZ > span:nth-child(5) > a"
element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
element.click()
time.sleep(2)

##결과검색 클릭
result_css_selector = "#_place_portal_root > div > div.B1gsM > div.qeVvz > a.GZe5x.Mj_l3"
result = driver.find_element(By.CSS_SELECTOR, result_css_selector)
result.click()
time.sleep(2)


#컨테이너박스 안에서 스크롤하기
container = driver.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container')
while True:
    # 현재 스크롤 위치
    last_height = driver.execute_script("return arguments[0].scrollTop;", container)
    # 컨테이너 아래로 스크롤
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", container)
    # 페이지 로딩 대기
    time.sleep(2)
    # 새로운 스크롤 위치
    new_height = driver.execute_script("return arguments[0].scrollTop;", container)
    # 스크롤이 더 이상 되지 않으면 반복 종료
    if new_height == last_height:
        break
        

# 스크롤 완료 후 검색 결과(식당) 가져오기
restaurant=[] #식당이름 저장 리스트 
for i in range(1, 51):
    try:
        element = driver.find_element(By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/a[1]/div/div/span[1]')
        text = element.text
        restaurant.append(text)
    except:
        print(f"No element found for index {i}")
        
    
#식당마다 리뷰 가져오기 
rev=[] #전체 식당 리뷰 저장 리스트
for i in range(1, 21):
    res_rev=[] #각각의 식당마다 리뷰 저장 리스트
    
    #첫번째 frame (검색창) 으로 옮김
    driver.switch_to.default_content()
    new_frame_xpath = '//*[@id="searchIframe"]'
    frame = driver.find_element(By.XPATH, new_frame_xpath)
    driver.switch_to.frame(frame)
    
    #식당 클릭, 해당 식당 frame으로 옮겨가기
    element_css_selector = "#_pcmap_list_scroll_container > ul > li:nth-child("+str(i)+") > div.CHC5F > a.tzwk0"
    element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
    element.click()
    time.sleep(2)
        
    #두번째 frame (해당 식당 frame)으로 옮김
    driver.switch_to.default_content()
    new_frame_xpath = '//*[@id="entryIframe"]'
    frame = driver.find_element(By.XPATH, new_frame_xpath)
    driver.switch_to.frame(frame)
    time.sleep(2)
        
    #리뷰 탭 클릭
    review_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "/restaurant/") and contains(@href, "/review?")]'))) #리뷰 위치 영향 안 받게 설정
    review_link.click()
    time.sleep(3)
        
    while True:
        try:
            # "더보기" 버튼을 찾고 클릭
            more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            more_button.click()
            # 리뷰 로드 대기 (10초 동안 기다림)
            time.sleep(5)
            # 현재까지 수집한 리뷰 개수 확인
            current_review_count = len(driver.find_elements(By.XPATH, '//ul[@class="eCPGL"]/li[@class="YeINN"]'))
            # 수집한 리뷰가 100개 이상이라면 반복 종료
            if current_review_count >= 100:
                break
        except Exception as e:
            # 더이상 "더보기" 버튼이 없거나 예외가 발생하면 반복 종료
            print("No more reviews or an error occurred.")
            break
        
        
    for i in range(1, 101): # 더보기 누르지 않은 상태로 최대 10개
        #두번째 frame으로 옮김
        driver.switch_to.default_content()
        new_frame_xpath = '//*[@id="entryIframe"]'
        frame = driver.find_element(By.XPATH, new_frame_xpath)
        driver.switch_to.frame(frame)
        time.sleep(4)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤 끝까지 내리기
        try:
            # 사진 없는 후기는 div.ZZ4OK.IwhtZ 클래스에 위치
            wait = WebDriverWait(driver, 5)
            try: #펼치기가 필요한 경우에는 텍스트를 눌러서 펼치기
                element_css_selector = '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child('+str(i)+') > div.ZZ4OK.IwhtZ > a > span.zPfVt'
                element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
                driver.execute_script("arguments[0].click();", element)
                time.sleep(2)
            except:pass
            #펼쳐진 리뷰들을 크롤링
            review_elements = wait.until(
                    EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="eCPGL"]/li[@class="YeINN"]')))
            review_text = review_elements[i].find_element(By.XPATH, './/span[@class="ZZ40K IwhtZ"]').text
            res_rev.append(review_text)
                
        except :
            try:
                # 사진 있는 후기는 div.ZZ4OK 클래스에 텍스트가 위치
                wait = WebDriverWait(driver, 5)
                try: #펼치기가 필요한 경우에는 텍스트를 눌러서 펼치기
                    element_css_selector = '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child('+str(i)+') > div.ZZ4OK > a > span.zPfVt'
                    element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
                    driver.execute_script("arguments[0].click();", element)
                    time.sleep(2)
                except:pass
                 #펼쳐진 리뷰들을 크롤링
                review_elements = wait.until(
                    EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="eCPGL"]/li[@class="YeINN"]')))
                review_text = review_elements[i-1].find_element(By.XPATH, './/span[@class="zPfVt"]').text
                res_rev.append(review_text)
                    
            except NoSuchElementException : #리뷰 텍스트가 없고 사진만 있는 경우, 결측치 처리
                res_rev.append('NAN')
                
    rev.append(res_rev) #각 식당의 리뷰를 전체 리뷰 리스트에 하나씩 넣기 




No element found for index 41
No element found for index 42
No element found for index 43
No element found for index 44
No element found for index 45
No element found for index 46
No element found for index 47
No element found for index 48
No element found for index 49
No element found for index 50


In [4]:
#식당명 리스트- 1페이지만 완료해서 50개임
restaurant

['봉성식당',
 '구이사이',
 'salon de lavant',
 '망고홀릭 애월해안도로점',
 '폴비앙',
 '애월 브런치오',
 '해변횟집',
 '바당한그릇',
 '심바카레',
 '꽁순이네',
 '투썸플레이스곽지과물해변점',
 '하르방짬뽕',
 '문동일셰프의 녹차고을',
 '시루애월',
 '제주돈아 본점',
 '제레미',
 '꽃밥',
 '보말보말',
 '제주할망밥상 애월점',
 '망고레이 본점',
 '파리바게뜨 제주하귀점',
 '남또리',
 '애월아시시',
 '제주빵집',
 'TONKATSU 서황',
 '화랑스시',
 '커피냅로스터스 제주',
 '황셰프양식당',
 '엄지흑돼지',
 '인디안 썸머 애월',
 '애월전분공장카페',
 '곽지국시',
 '앙데팡당',
 '명리동식당 애월점',
 '오복수산 애월점',
 '참솔식당',
 '골목카페옥수',
 '카페포메',
 '이순신김밥',
 '참좋은해물라면 애월']

In [5]:
#전체 리뷰 리스트-우선 시간 상 1개식당 * 각 100개 리뷰 (더보기 안 누름) 만 해본 결과 
rev

[['베이컨에그롤샐러드 맛있었어요. 드레싱도 상큼하고 야채도 신선,올리브와 드라이토마토, 치즈.. 토핑까지 다 좋았고 베이컨에그롤이 정말 맛있었어요. 커피도 디카페인이 가능해서 매우 만족. 크림치즈애플팬케익은 크림치즈가 아니라 일반 치즈 같았는데 부드럽고 괜찮았어요. 초코라떼 많이 달지 않아 좋았습니다. 카페 분위기 깔끔하고 세련된 느낌이에요.',
  '커피도 맛있고 \n샐러드 소스, 빵 굽기정도, 감자스프 모든게 완벽해요❤️\n\n분위기가 너무 좋고 사장님의 인상과 너무 잘 어울리는 인테리어였어요👍\n키우시는 강아지도 너무 예쁘고 \n길고양이도 개냥이라 힐링하고 갑니다 :)',
  '커피도 맛있고 디저트도 맛있어서 초창기때부터 꾸준히 가는 곳이예요~ 후기는 처음이네요!\n분위기, 맛, 뭐하나 빠지는거 없는 참 좋은 곳입니다',
  '조용한 마을에 위치한 숨어있는 곳이에요.~\n카페 분위기는 전반적으로 차분하고, 조용해요.\n단골 손님이 꾀 있어보였어요.~ 큰 멍멍이도 함께 있는 곳입니다.\n사장님께서 취미로 그림도 그리신데요, 곳곳에 작품들도\n비치되어있어요.\n여유있게 잘 보내고 갑니다.^^',
  '조용한 동네의 차분한 분위기가 좋고 강아지와 고양이가 있어서 더 좋아요^^ 팬케이크도 넘 맛있네요~',
  '커피가 맛있어요\n번잡하지 않고 조용히 대화하기 좋네요',
  '인테리어가 크리스마스 느낌 물씬나요 커피도 맛있어요',
  '커피 산미 고소한것도 있었음 좋겠어요',
  '분위기 너무 좋고 팬케이크도 맛있었어요! 📌',
  '구우우우웃~',
  '셀러드가 기가막히네요',
  '밥먹으러 갔다가ㅡ 근처 카페 검색해서 찾아감. 새로운 곳을 찾아가는 재미.. 작은 까페지만 정원도 사진찍기 이쁘게 아담하게 가꿔놓음. 디저트는 팬케이크 추천해주셨는데ㅡ 맛이 없었음. 너무 퍽퍽했음ㅠ 같이간 일행하고 맛없다...하며 먹음~;; 커피는 고소하고 좋았음! 주차는 큰차는 위치에 따라 애매 할 수 있으나 나쁘지않음. 화장실은 공용이라 그게 넘 별로였음. 어쩔수없이 앞에 남자

In [14]:
from selenium.common.exceptions import NoSuchElementException

driver_path = 'C:/Users/임채현/[숙탯]겨울/chromedriver.exe'

# ChromeOptions 설정
chrome_options = webdriver.ChromeOptions()


# 다른 옵션들을 추가하려면 여기에 추가
#service = Service(executable_path=driver_path)

# 크롬 드라이버 초기화
driver = webdriver.Chrome( options=chrome_options)
driver.get('https://map.naver.com/v5/search')

# 팝업 창 제거
#driver.find_element_by_css_selector("button#intro_popup_close").click()


time.sleep(5)


#애월맛집 검색, 맛집 리스트 뽑기
search_input = driver.find_element(By.CSS_SELECTOR, 'div.input_box input.input_search')
search_input.click()
search_input.send_keys("애월맛집")
time.sleep(3)

search_input.send_keys(Keys.ENTER)
time.sleep(5)

#프레임 변경
frame_xpath = '//*[@id="searchIframe"]'
frame = WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, frame_xpath))
)

#필터링 - 리뷰 많은 순으로 설정
##필터링 버튼 클릭
fil_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div[1]/div/div/div/div/div/span[1]/a'))
    )
fil_button.click()
time.sleep(5)

##리뷰많은순 클릭
element_css_selector = "#_place_portal_root > div > div.B1gsM > div.YWnDg > div > div > div:nth-child(1) > div.RhjQZ > span:nth-child(5) > a"
element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
element.click()
time.sleep(2)

##결과검색 클릭
result_css_selector = "#_place_portal_root > div > div.B1gsM > div.qeVvz > a.GZe5x.Mj_l3"
result = driver.find_element(By.CSS_SELECTOR, result_css_selector)
result.click()
time.sleep(2)

##다음페이지로 넘김 - 넘겨야되는 경우만!
element_css_selector = "#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(5)"
element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
element.click()
time.sleep(3)

#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(2)
#app-root > div > div.XUrfU > div.zRM9F > a.mBN2s.qxokY


#컨테이너박스 안에서 스크롤하기
container = driver.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container')
while True:
    # 현재 스크롤 위치
    last_height = driver.execute_script("return arguments[0].scrollTop;", container)
    # 컨테이너 아래로 스크롤
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", container)
    # 페이지 로딩 대기
    time.sleep(2)
    # 새로운 스크롤 위치
    new_height = driver.execute_script("return arguments[0].scrollTop;", container)
    # 스크롤이 더 이상 되지 않으면 반복 종료
    if new_height == last_height:
        break
        

# 스크롤 완료 후 검색 결과(식당) 가져오기
restaurant=[] #식당이름 저장 리스트 
for i in range(1, 11):
    try:
        element = driver.find_element(By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/a[1]/div/div/span[1]')
        text = element.text
        restaurant.append(text)
    except:
        print(f"No element found for index {i}")
        
    
#식당마다 리뷰 가져오기 
rev=[] #전체 식당 리뷰 저장 리스트
for i in range(1, 11):
    res_rev=[] #각각의 식당마다 리뷰 저장 리스트
    
    #첫번째 frame (검색창) 으로 옮김
    driver.switch_to.default_content()
    new_frame_xpath = '//*[@id="searchIframe"]'
    frame = driver.find_element(By.XPATH, new_frame_xpath)
    driver.switch_to.frame(frame)
    
    #식당 클릭, 해당 식당 frame으로 옮겨가기
    element_css_selector = "#_pcmap_list_scroll_container > ul > li:nth-child("+str(i)+") > div.CHC5F > a.tzwk0"
    element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
    element.click()
    time.sleep(2)
        
    #두번째 frame (해당 식당 frame)으로 옮김
    driver.switch_to.default_content()
    new_frame_xpath = '//*[@id="entryIframe"]'
    frame = driver.find_element(By.XPATH, new_frame_xpath)
    driver.switch_to.frame(frame)
    time.sleep(2)
        
    #리뷰 탭 클릭
    review_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "/restaurant/") and contains(@href, "/review?")]'))) #리뷰 위치 영향 안 받게 설정
    review_link.click()
    time.sleep(3)
        
    while True:
        try:
            # "더보기" 버튼을 찾고 클릭
            more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            more_button.click()
            # 리뷰 로드 대기 (10초 동안 기다림)
            time.sleep(5)
            # 현재까지 수집한 리뷰 개수 확인
            current_review_count = len(driver.find_elements(By.XPATH, '//ul[@class="eCPGL"]/li[@class="YeINN"]'))
            # 수집한 리뷰가 100개 이상이라면 반복 종료
            if current_review_count >= 100:
                break
        except Exception as e:
            # 더이상 "더보기" 버튼이 없거나 예외가 발생하면 반복 종료
            print("No more reviews or an error occurred.")
            break
        
        
    for i in range(1, 101): # 더보기 누르지 않은 상태로 최대 10개
        #두번째 frame으로 옮김
        driver.switch_to.default_content()
        new_frame_xpath = '//*[@id="entryIframe"]'
        frame = driver.find_element(By.XPATH, new_frame_xpath)
        driver.switch_to.frame(frame)
        time.sleep(4)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤 끝까지 내리기
        try:
            # 사진 없는 후기는 div.ZZ4OK.IwhtZ 클래스에 위치
            wait = WebDriverWait(driver, 5)
            try: #펼치기가 필요한 경우에는 텍스트를 눌러서 펼치기
                element_css_selector = '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child('+str(i)+') > div.ZZ4OK.IwhtZ > a > span.zPfVt'
                element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
                driver.execute_script("arguments[0].click();", element)
                time.sleep(2)
            except:pass
            #펼쳐진 리뷰들을 크롤링
            review_elements = wait.until(
                    EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="eCPGL"]/li[@class="YeINN"]')))
            review_text = review_elements[i-1].find_element(By.XPATH, './/span[@class="ZZ40K IwhtZ"]').text
            res_rev.append(review_text)
                
        except :
            try:
                # 사진 있는 후기는 div.ZZ4OK 클래스에 텍스트가 위치
                wait = WebDriverWait(driver, 5)
                try: #펼치기가 필요한 경우에는 텍스트를 눌러서 펼치기
                    element_css_selector = '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child('+str(i)+') > div.ZZ4OK > a > span.zPfVt'
                    element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
                    driver.execute_script("arguments[0].click();", element)
                    time.sleep(2)
                except:pass
                 #펼쳐진 리뷰들을 크롤링
                review_elements = wait.until(
                    EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="eCPGL"]/li[@class="YeINN"]')))
                review_text = review_elements[i-1].find_element(By.XPATH, './/span[@class="zPfVt"]').text
                res_rev.append(review_text)
                    
            except NoSuchElementException : #리뷰 텍스트가 없고 사진만 있는 경우, 결측치 처리
                res_rev.append('NAN')
                
    rev.append(res_rev) #각 식당의 리뷰를 전체 리뷰 리스트에 하나씩 넣기 




No more reviews or an error occurred.


WebDriverException: Message: unknown error: cannot deserialize the result value received from Runtime.callFunctionOn
  (Session info: chrome=121.0.6167.140)


In [ ]:
#식당명 리스트- 1페이지만 완료해서 50개임
restaurant

In [12]:
#전체 리뷰 리스트-우선 시간 상 1개식당 * 각 100개 리뷰 (더보기 안 누름) 만 해본 결과 
rev

[['제주도왔으니 흙돼지 맛집 한번와봤습니당\n여기는 고사리가 특별한메뉴로 나오는듯해요!\n추가할시 5천원추가금있어요!\n4명이서 3인분했는데도 배불리먹었네용ㅎㅎ\n된장찌개는 게가들어가서 시원한맛이에요~\n사장님너무친절하시고 직접구워주시고 설명도 해주셔서 먹기편했답니당\n주차는 가게옆에 두 대 가능하지만 대충 아무데나 주차해도되는게 제주도의 편리한점아닐까요..ㅎ',
  '유명하다고 해서 방문했는데 이유가 있었어요 :) 흑돼지 고기도 질이 좋고 너무 맛있었어요 주차 2대 정도 할 수 있습니다! 셀프바도 깔끔합니다 고사리 기본으로 나오고 오천원 추가 가능해요~',
  '유튜브에서 보고 유명하길래 찜 해뒀었는데 숙소 근처 걸어서 2분 거리에 있길래 오픈런 해서 다녀왔어요!!\n기대한만큼 만족스러웠던 곳이 몇 없었는데 여긴 사장님이 들어가자마자 따뜻하게 인사해주시고 구워주셔서 차분히 음미하면서 먹을 수 있었는데 천천히 먹으라며 신경 써주시구요!\n고기질은 굽기 전부터 대박이다 싶었는데 먹자마자 와...했습니다👍🏻오겹살이 진짜 쩔어요 고사리랑 백묵은지도 구워주시는데 곁들여 먹으니까 진짜 술술 넘어가더라구요 지칠 줄을 모르고 밥 한공기에 뚝딱했습니다!!반찬도 안 맛있는 게 없었어요 진짜...하 또 먹고싶다 다음에도 제주오면 여기 꼭 재방문 할 거예요 계산해주신 남자 사장님은 웃으며 허리 숙여 인사하시면서 카드 두손으로 주실 때 진짜 감동했어요\U0001f979너무 기분좋은 식사를 해서 행복했습니다^^',
  '우연히 방문했는데 맛집이었나봐요 .\n손님이 엄청 많았음!! \n1인분단위가 아닌 근단위라 둘이가니 양이 많았고,\n구워주셔서 좋아요',
  '흑돼지 맛집이라고 유명하다고해서 갔는데 진짜 맛있었어요. 다른것보다 직접 키운 고사리를 구워먹는게 신박하고, 먹기 알맞게 직접 구워주셔서 좋았어요.',
  '#제주\n#맛집 #봉성식당 #애월\n#고기맛집\n\n제주 가면 늘 가는 식당이예요\n고기 넘 ~ 맛나요 ~~!! \n재주 가면 꼭 ~ 들러보세요 ~ ^^',
  '숙박 

In [15]:
#전체 리뷰 리스트-우선 시간 상 1개식당 * 각 100개 리뷰 (더보기 안 누름) 만 해본 결과 
rev

[['제주도왔으니 흙돼지 맛집 한번와봤습니당\n여기는 고사리가 특별한메뉴로 나오는듯해요!\n추가할시 5천원추가금있어요!\n4명이서 3인분했는데도 배불리먹었네용ㅎㅎ\n된장찌개는 게가들어가서 시원한맛이에요~\n사장님너무친절하시고 직접구워주시고 설명도 해주셔서 먹기편했답니당\n주차는 가게옆에 두 대 가능하지만 대충 아무데나 주차해도되는게 제주도의 편리한점아닐까요..ㅎ',
  '유명하다고 해서 방문했는데 이유가 있었어요 :) 흑돼지 고기도 질이 좋고 너무 맛있었어요 주차 2대 정도 할 수 있습니다! 셀프바도 깔끔합니다 고사리 기본으로 나오고 오천원 추가 가능해요~',
  '유튜브에서 보고 유명하길래 찜 해뒀었는데 숙소 근처 걸어서 2분 거리에 있길래 오픈런 해서 다녀왔어요!!\n기대한만큼 만족스러웠던 곳이 몇 없었는데 여긴 사장님이 들어가자마자 따뜻하게 인사해주시고 구워주셔서 차분히 음미하면서 먹을 수 있었는데 천천히 먹으라며 신경 써주시구요!\n고기질은 굽기 전부터 대박이다 싶었는데 먹자마자 와...했습니다👍🏻오겹살이 진짜 쩔어요 고사리랑 백묵은지도 구워주시는데 곁들여 먹으니까 진짜 술술 넘어가더라구요 지칠 줄을 모르고 밥 한공기에 뚝딱했습니다!!반찬도 안 맛있는 게 없었어요 진짜...하 또 먹고싶다 다음에도 제주오면 여기 꼭 재방문 할 거예요 계산해주신 남자 사장님은 웃으며 허리 숙여 인사하시면서 카드 두손으로 주실 때 진짜 감동했어요\U0001f979너무 기분좋은 식사를 해서 행복했습니다^^',
  '우연히 방문했는데 맛집이었나봐요 .\n손님이 엄청 많았음!! \n1인분단위가 아닌 근단위라 둘이가니 양이 많았고,\n구워주셔서 좋아요',
  '흑돼지 맛집이라고 유명하다고해서 갔는데 진짜 맛있었어요. 다른것보다 직접 키운 고사리를 구워먹는게 신박하고, 먹기 알맞게 직접 구워주셔서 좋았어요.',
  '#제주\n#맛집 #봉성식당 #애월\n#고기맛집\n\n제주 가면 늘 가는 식당이예요\n고기 넘 ~ 맛나요 ~~!! \n재주 가면 꼭 ~ 들러보세요 ~ ^^',
  '숙박 

In [ ]:
#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(7) > svg
#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(5)

In [17]:
res_1_10 = rev
res_1_10

[['제주도왔으니 흙돼지 맛집 한번와봤습니당\n여기는 고사리가 특별한메뉴로 나오는듯해요!\n추가할시 5천원추가금있어요!\n4명이서 3인분했는데도 배불리먹었네용ㅎㅎ\n된장찌개는 게가들어가서 시원한맛이에요~\n사장님너무친절하시고 직접구워주시고 설명도 해주셔서 먹기편했답니당\n주차는 가게옆에 두 대 가능하지만 대충 아무데나 주차해도되는게 제주도의 편리한점아닐까요..ㅎ',
  '유명하다고 해서 방문했는데 이유가 있었어요 :) 흑돼지 고기도 질이 좋고 너무 맛있었어요 주차 2대 정도 할 수 있습니다! 셀프바도 깔끔합니다 고사리 기본으로 나오고 오천원 추가 가능해요~',
  '유튜브에서 보고 유명하길래 찜 해뒀었는데 숙소 근처 걸어서 2분 거리에 있길래 오픈런 해서 다녀왔어요!!\n기대한만큼 만족스러웠던 곳이 몇 없었는데 여긴 사장님이 들어가자마자 따뜻하게 인사해주시고 구워주셔서 차분히 음미하면서 먹을 수 있었는데 천천히 먹으라며 신경 써주시구요!\n고기질은 굽기 전부터 대박이다 싶었는데 먹자마자 와...했습니다👍🏻오겹살이 진짜 쩔어요 고사리랑 백묵은지도 구워주시는데 곁들여 먹으니까 진짜 술술 넘어가더라구요 지칠 줄을 모르고 밥 한공기에 뚝딱했습니다!!반찬도 안 맛있는 게 없었어요 진짜...하 또 먹고싶다 다음에도 제주오면 여기 꼭 재방문 할 거예요 계산해주신 남자 사장님은 웃으며 허리 숙여 인사하시면서 카드 두손으로 주실 때 진짜 감동했어요\U0001f979너무 기분좋은 식사를 해서 행복했습니다^^',
  '우연히 방문했는데 맛집이었나봐요 .\n손님이 엄청 많았음!! \n1인분단위가 아닌 근단위라 둘이가니 양이 많았고,\n구워주셔서 좋아요',
  '흑돼지 맛집이라고 유명하다고해서 갔는데 진짜 맛있었어요. 다른것보다 직접 키운 고사리를 구워먹는게 신박하고, 먹기 알맞게 직접 구워주셔서 좋았어요.',
  '#제주\n#맛집 #봉성식당 #애월\n#고기맛집\n\n제주 가면 늘 가는 식당이예요\n고기 넘 ~ 맛나요 ~~!! \n재주 가면 꼭 ~ 들러보세요 ~ ^^',
  '숙박 

In [18]:
import pandas as pd

In [20]:
df_1_10 = pd.DataFrame(res_1_10)
df_1_10

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,제주도왔으니 흙돼지 맛집 한번와봤습니당\n여기는 고사리가 특별한메뉴로 나오는듯해요!...,유명하다고 해서 방문했는데 이유가 있었어요 :) 흑돼지 고기도 질이 좋고 너무 맛있...,유튜브에서 보고 유명하길래 찜 해뒀었는데 숙소 근처 걸어서 2분 거리에 있길래 오픈...,우연히 방문했는데 맛집이었나봐요 .\n손님이 엄청 많았음!! \n1인분단위가 아닌 ...,흑돼지 맛집이라고 유명하다고해서 갔는데 진짜 맛있었어요. 다른것보다 직접 키운 고사...,#제주\n#맛집 #봉성식당 #애월\n#고기맛집\n\n제주 가면 늘 가는 식당이예요\...,숙박 근처라 다녀왔는데 친절하셔서 더 맛있게 먹었어요. 감사합니다,제주 찐 맛집입니다 제주 오시는 분 필수 방문 요망!!!,제주도 왔으니 흑돼지 먹어야죠 !!\n너무 기대해서 그런가 맛있긴하지만 그렇게 특별...,사장님들 너무 친절하시고 바쁘신 와중에도 테이블 계속 신경 써주셔서 편안하고 맛있게...,...,흑돼지도 흑돼지지만 고사리도 구워 먹고 싶어서 간 곳인데 맛있었어요!,1인분이 300g이예요,내 마음을 뺏어간 절미강아지\n절미에 이끌려 따라갔는데 \n연예인들 많이 다녀간 맛...,NAN,좋아요,진짜쫀쫀맛,찐현지인 맛집입니다!\n제주 올때마다 들러요!\n고기는 당연히 맛있고\n후식김치찌개...,너무 맛있네요\n서울에서 고깃집하는 사장입니다.\n\n오랜만에 맛있는 흑돼지집 찾은...,너무 맛있어요!!,"여러 흑돼지 맛집들 웨이팅을 이겨낼 자신이 없어서,, 도민 맛집으로 유명하다는 봉성..."
1,"아버지와 여행중에 방문했어요. 특정식(15,000원/1인) 2인 주문해서 먹었어요....",전체적으로 모든 반찬이 밥과 어울리고 생선 또한 고소해서 오길잘했다 라는 생각이 백...,무려 게장이 밑반찬인 현지인맛집 이에요 ㅎㅎ 흑돼지 백돼지 둘다 먹을 수 있어서 좋...,아니.. 여기 유명해지면 좋겠는데 안유명했으면 좋겠는 제 모순된 마음.. 고기가 정...,정식 종류도 많고 맛있었어요 ㅎ 사장님도 친절하셔서 또 방문하고 싶은 곳이에오,참 괜찮은 식당이네요\n애월에 머무는 동안은 아침식사하러 오기 좋을것 같아요\n친절...,기본찬들이 맛있고 양이 많습니다.\n고등어 구이가 부드럽고 정말 맛있어요.,"와 여기 진짜 맛있어요! 저희는 구이사이 모듬 중으로 시켰는데 항정, 오겹, 뼈갈비...",생선구이 좋아하는 아들 때문에 검색해서 방문한 구이사이...결과는..대만족이에요!!...,애월에 아침가능 한곳이라서 처음에 반신반의 하는 마음으로 갔는데 2번 연속으로 가서...,...,푸짐푸짐!\n오늘 점심 정말 넘 맛있게 먹고왔어요~\n어느하나 콕집어 말하지 않아도...,음식이 아주 맛나네요^^ 다음에도 또 오고싶어요~,음식이 좀 늦게 나와서 기다렸지만\n그래도 맛있었던 한정식!,제주도 물가에 이런가격에 이렇게 푸짐한 구성으로 먹을 수 있다는게 놀랐어요 관광객 ...,제주도 가족여행 4일차\n집밥이 그리운던차에 이런 맛집을~\n집밥좋아하는 우리집 아...,넘 맛있게 잘먹었습니다,로컬맛집이라고 해서 들렸는데. 이미 유명해져서.ㅋ\n가격대는 높은편이다. 추가메뉴가...,맛있어요,오랫만에 구이사이🤍\n정식은 9242,김치찌게와 고등어구이 .. 한식을 좋아하는\n아이들이라 아이들도 잘먹고 저두 배가 ...
2,"베이컨에그롤샐러드 맛있었어요. 드레싱도 상큼하고 야채도 신선,올리브와 드라이토마토,...","커피도 맛있고 \n샐러드 소스, 빵 굽기정도, 감자스프 모든게 완벽해요❤️\n\n분...",커피도 맛있고 디저트도 맛있어서 초창기때부터 꾸준히 가는 곳이예요~ 후기는 처음이네...,"조용한 마을에 위치한 숨어있는 곳이에요.~\n카페 분위기는 전반적으로 차분하고, 조...",조용한 동네의 차분한 분위기가 좋고 강아지와 고양이가 있어서 더 좋아요^^ 팬케이크...,커피가 맛있어요\n번잡하지 않고 조용히 대화하기 좋네요,인테리어가 크리스마스 느낌 물씬나요 커피도 맛있어요,커피 산미 고소한것도 있었음 좋겠어요,분위기 너무 좋고 팬케이크도 맛있었어요! 📌,구우우우웃~,...,맛있어요,음악도좋고편안한분위기는더좋고팬케익도 커피도맛나네요,완전 맛있어요!!!!!!!!!!,좋습니다 ^^ 재방문 의사있음,팬케이크도 맛있고 커피도 맛있고 흐린날씨였지만 간간이 햇살 들어오는 가게도 예쁘고 ...,굿,맛나유,팬케이크 맛집이라고 해서 갔는데 너~무 맛있어요! 커피도 맛있고 다 맛있어서 좋았습니다!,팬케익 맛있음,아주 오래전 기억이 너무 좋아서 다시 찾았어요~\n역시는 역시입니다!!!\n담엔 팬...
3,"혼자 시간떼우러 들어왓다가 우연찮게 들어왓는데,,\n빵도 너무 맛잇고 커피도 너무 ...",아이바가든 왔다가 베이커리랑 차마시러 왔음 매장 넓고 빵도 초코거북이빵 귀엽개 생김...,"베이커리 , 음료 맛있는 집이라 \n친구랑 다녀왔는데요 \n\n한라봉 스무디도 주문...",평화로 길가에 있는 대형카페에요\n바다도 보이고 가게 인테리어가 너무 이쁩니다\n빵...,집근처에 아이바가든이 있어서 꼭 와봐야지 했는데\n여기 너무 멋진 카페도 있네요~~...,아이바가든을 가기위해서 왔다가 바로 옆에 베이커리카페가 있길래 들어왔어요 !\n매장...,아이바 가든 구경 후 들린 폴비앙 카페!\n유럽풍 인테리어가 인상적이었고 층고가 높...,2024년 새해를 맞이하여 집에만 있기엔 아쉬워서 훌쩍떠난 카페 투어~ 책도 보고 ...,아이바가든~^^예약하면서 음료같이 예약했어요ㆍ웬열~~가성비갑~^^완죤득템. 만족만족...,눈이 많이 와서 비행기 결항으로 제주도에 더 머물게 되었어요. 날씨가 좋지 않아 실...,...,저는 오늘 저의 아이들과 함께 카페에 왔습니다.음료를 시켰는데 직원분도 친절하시고 ...,폴비앙~ 아이바가든 들리고난후 폴비앙카페 들렸다. 버터 풍미가 가득한 프랑스전통 크...,다른나라여행와서 잠시쉬다가는 카페같아요\n볼거리도너무많고 빵 완전진짜맛있어요!!\n...,"아이랑 어디갈가 찾다가 발견한 폴비앙 카페! 매장이 넓고 아기의자가 있어서 왔는데,...",아이바가든 갔다가 들렀는데\n매장이 넓고 좋네요~~\n출출한김에 빵까지~~ 굳\n아...,"베이커리랑 반려견 동반까지 가능한 카페 찾다가 발견 했는데, 제주도에서 온 카페 중...",우연히 지나가다 들렸는데 일단 분위기가 마음에 들고 음료랑 빵이 맛있어요~~원래 리...,부모님 칠순기념으로 가족여행 왔다가\n모닝커피 하러 왔어요\n커피랑 코코아 맛있고요...,생각보다 커서 놀랬습니다\n아이바가든 입장권과 함께 카페 이용권구입하여 저렴하게 먹...,부모님이랑 함께 찾은 폴비앙\n뷰와 인테리어가 멋진곳...ㅎㅎ\n다들 좋아하셔서 지...
4,제주도 오셨으면 망고빙수는 꼭 먹어줘야죠?\n애월 해안도로 바로 앞에 있어서 뷰도 ...,"와 존맛탱,,, 생과일 망고 진짜 너무 맛있구요 아래 깔려있는 눈꽃빙수며, 코코넛이...",냉동이 아닌 생망고로 빙수를 먹다니 너무 좋으네요 ~ 애월해안도로에 위치해있어서 뷰...,분위기부터 동남아 여행 온 느낌이 가득 나요!!\n그리고 빙수도 많이 안달고 너무 ...,해안도로에서 사진 찍으려고 차를 세우다 발견한 망고 홀릭카페^^ 노란색 포인트와 이...,"대설 한파를 뚫고 제주도 여행 온 나란 사람,,,\n빙수가 먹고 싶어서 슬쩍 들렸는...",애월카페 망고홀릭 애월해안로점 에 아이들과 다녀왔어요.달콤함에 반하고 친절에 더 반...,망고 너무 좋아하는데 넘 맛있게 먹었어요!!!\n이국적인 가게 분위기도 너무 재밌어...,본점에서 너무 맛있게 먹어서 애월점에도 방문!!\n애월점은 바다 바로 앞이라 뷰가 ...,진짜 제주도 도착하자마자 달려온 곳이에요~~ 여름에 먹었을 때 너무 맛있어서 또 왔...,...,생망고 빙수 너무너무 맛있어요!!!! 그냥 토핑없 망고랑 얼음만 먹어도 맛있고 선물...,망고빙수 망고주스 진짜 신선하고 맛있어요\n냉동 아니고 생과를 올려놔서 더 좋았어요...,맛있어서 또 왔어요 담에 오면 또또 얼 거에요 너무 달지도 않고 딱 조아요🙂,뷰도 좋고 망고 빙수 정말 맛있어요,아이들이 망고좋아해서 들른곳이예요. 아이들은 아직 차가운걸 못먹어서 망고만 골라먹네...,망

In [21]:
df_1_10.transpose()

,0,1,2,3,4,5,6
0,제주도왔으니 흙돼지 맛집 한번와봤습니당\n여기는 고사리가 특별한메뉴로 나오는듯해요!...,"아버지와 여행중에 방문했어요. 특정식(15,000원/1인) 2인 주문해서 먹었어요....","베이컨에그롤샐러드 맛있었어요. 드레싱도 상큼하고 야채도 신선,올리브와 드라이토마토,...","혼자 시간떼우러 들어왓다가 우연찮게 들어왓는데,,\n빵도 너무 맛잇고 커피도 너무 ...",제주도 오셨으면 망고빙수는 꼭 먹어줘야죠?\n애월 해안도로 바로 앞에 있어서 뷰도 ...,"애월해안도로에 새로 생긴 브런치카페에요!\n주차공간도 넓고 내부도 넓고, 바다가 한...",제주 여행을 온지 2일차 되는날에 방문했습니다.\n배타고 낚시해서 물고기를 꽤 잡았...
1,유명하다고 해서 방문했는데 이유가 있었어요 :) 흑돼지 고기도 질이 좋고 너무 맛있...,전체적으로 모든 반찬이 밥과 어울리고 생선 또한 고소해서 오길잘했다 라는 생각이 백...,"커피도 맛있고 \n샐러드 소스, 빵 굽기정도, 감자스프 모든게 완벽해요❤️\n\n분...",아이바가든 왔다가 베이커리랑 차마시러 왔음 매장 넓고 빵도 초코거북이빵 귀엽개 생김...,"와 존맛탱,,, 생과일 망고 진짜 너무 맛있구요 아래 깔려있는 눈꽃빙수며, 코코넛이...",오션뷰카페를 검색해서 우연히 발견한 브런치오! 오픈한지 얼마 안됐는지 가게 내부 깔...,여기 사장님 맛잘알이세요~!!!\n김에 양파절임 양념밥 고등어회 싸먹는 조합이 너무...
2,유튜브에서 보고 유명하길래 찜 해뒀었는데 숙소 근처 걸어서 2분 거리에 있길래 오픈...,무려 게장이 밑반찬인 현지인맛집 이에요 ㅎㅎ 흑돼지 백돼지 둘다 먹을 수 있어서 좋...,커피도 맛있고 디저트도 맛있어서 초창기때부터 꾸준히 가는 곳이예요~ 후기는 처음이네...,"베이커리 , 음료 맛있는 집이라 \n친구랑 다녀왔는데요 \n\n한라봉 스무디도 주문...",냉동이 아닌 생망고로 빙수를 먹다니 너무 좋으네요 ~ 애월해안도로에 위치해있어서 뷰...,파스타에 참나물 향이 이렇게 잘 어울릴줄이야~\n커피도 쓰지 않고 부드럽게 잘 넘어...,아이들도 어른들도 모두 만족하는 집이네요~\n맛집을발견해서 혼자만 알고싶었는데ㅜ\n...
3,우연히 방문했는데 맛집이었나봐요 .\n손님이 엄청 많았음!! \n1인분단위가 아닌 ...,아니.. 여기 유명해지면 좋겠는데 안유명했으면 좋겠는 제 모순된 마음.. 고기가 정...,"조용한 마을에 위치한 숨어있는 곳이에요.~\n카페 분위기는 전반적으로 차분하고, 조...",평화로 길가에 있는 대형카페에요\n바다도 보이고 가게 인테리어가 너무 이쁩니다\n빵...,분위기부터 동남아 여행 온 느낌이 가득 나요!!\n그리고 빙수도 많이 안달고 너무 ...,저번주에 이어 또 왔어요! 커피도 브런치도 너무 맛있어요:) 오늘은 날씨까지 좋아 ...,모듬 시키면 매운탕도 나와서 좋아요!\n솔직하게 회가 엄청 미친듯이 맛있는건 아닌데...
4,흑돼지 맛집이라고 유명하다고해서 갔는데 진짜 맛있었어요. 다른것보다 직접 키운 고사...,정식 종류도 많고 맛있었어요 ㅎ 사장님도 친절하셔서 또 방문하고 싶은 곳이에오,조용한 동네의 차분한 분위기가 좋고 강아지와 고양이가 있어서 더 좋아요^^ 팬케이크...,집근처에 아이바가든이 있어서 꼭 와봐야지 했는데\n여기 너무 멋진 카페도 있네요~~...,해안도로에서 사진 찍으려고 차를 세우다 발견한 망고 홀릭카페^^ 노란색 포인트와 이...,뇨끼도 정말 쫀득하니 맛있었고 크림소스가 버섯과 마늘프레이크와 잘 어우러져 깊은 맛...,이 집 오려고 경북 안동에서 왔어요!!!\nㅎㅎ고등어 회 퀄리티!! 소주가 생각나는...
...,...,...,...,...,...,...,...
95,진짜쫀쫀맛,넘 맛있게 잘먹었습니다,굿,"베이커리랑 반려견 동반까지 가능한 카페 찾다가 발견 했는데, 제주도에서 온 카페 중...",망고쥬스가 먹고싶어서 검색하다가 찾아왔어요 \n빙수양도 많고 드라이아이스 해서 아이...,아침안먹고 바로 여행가다가 꼬르륵거려서 방문 ㅋㅋㅋ 브런치 너무좋았어요,고등어회 처음 먹었는데 하나도 안 비리고 너무 맛있었어요 둘인데 식사 안할거면 1인...
96,찐현지인 맛집입니다!\n제주 올때마다 들러요!\n고기는 당연히 맛있고\n후식김치찌개...,로컬맛집이라고 해서 들렸는데. 이미 유명해져서.ㅋ\n가격대는 높은편이다. 추가메뉴가...,맛나유,우연히 지나가다 들렸는데 일단 분위기가 마음에 들고 음료랑 빵이 맛있어요~~원래 리...,망고빙수안에 톡톡 터지는게 너무 맛있어요 담에 꼭 또 올거에요 꼭이요,뷰가 완전 좋네요,NAN
97,너무 맛있네요\n서울에서 고깃집하는 사장입니다.\n\n오랜만에 맛있는 흑돼지집 찾은...,맛있어요,팬케이크 맛집이라고 해서 갔는데 너~무 맛있어요! 커피도 맛있고 다 맛있어서 좋았습니다!,부모님 칠순기념으로 가족여행 왔다가\n모닝커피 하러 왔어요\n커피랑 코코아 맛있고요...,연기가 나서 놀랬음 ㅋ\n팥이랑 아이스크림이랑은 곁들어 먹는 거 같음\n빙수가 맛있...,커피가 맛있어요~^^,고등어회 3~4인분짜리를 포장해와서 4인이 저녁으로 먹었음. 매운탕도 같이 포장해줌...
98,너무 맛있어요!!,오랫만에 구이사이🤍\n정식은 9242,팬케익 맛있음,생각보다 커서 놀랬습니다\n아이바가든 입장권과 함께 카페 이용권구입하여 저렴하게 먹...,제주여행 중 더위에 지쳐 근처 망고빙수를 검색하다가 우연히 찾아 온 곳 ! 기대는 ...,너무너무 매력있어요~ 애월해안로 가다 만난곳인데 재방문무조건하고싶어요,굿 맛있어요~


In [22]:
df_1_10.to_csv('rev_1_7.csv',encoding='utf-8-sig')

In [23]:
from selenium.common.exceptions import NoSuchElementException

driver_path = 'C:/Users/임채현/[숙탯]겨울/chromedriver.exe'

# ChromeOptions 설정
chrome_options = webdriver.ChromeOptions()


# 다른 옵션들을 추가하려면 여기에 추가
#service = Service(executable_path=driver_path)

# 크롬 드라이버 초기화
driver = webdriver.Chrome( options=chrome_options)
driver.get('https://map.naver.com/v5/search')

# 팝업 창 제거
#driver.find_element_by_css_selector("button#intro_popup_close").click()


time.sleep(5)


#애월맛집 검색, 맛집 리스트 뽑기
search_input = driver.find_element(By.CSS_SELECTOR, 'div.input_box input.input_search')
search_input.click()
search_input.send_keys("애월맛집")
time.sleep(3)

search_input.send_keys(Keys.ENTER)
time.sleep(5)

#프레임 변경
frame_xpath = '//*[@id="searchIframe"]'
frame = WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, frame_xpath))
)

#필터링 - 리뷰 많은 순으로 설정
##필터링 버튼 클릭
fil_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div[1]/div/div/div/div/div/span[1]/a'))
    )
fil_button.click()
time.sleep(5)

##리뷰많은순 클릭
element_css_selector = "#_place_portal_root > div > div.B1gsM > div.YWnDg > div > div > div:nth-child(1) > div.RhjQZ > span:nth-child(5) > a"
element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
element.click()
time.sleep(2)

##결과검색 클릭
result_css_selector = "#_place_portal_root > div > div.B1gsM > div.qeVvz > a.GZe5x.Mj_l3"
result = driver.find_element(By.CSS_SELECTOR, result_css_selector)
result.click()
time.sleep(2)

##다음페이지로 넘김 - 넘겨야되는 경우만!
element_css_selector = "#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(5)"
element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
element.click()
time.sleep(3)

#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(2)
#app-root > div > div.XUrfU > div.zRM9F > a.mBN2s.qxokY


#컨테이너박스 안에서 스크롤하기
container = driver.find_element(By.CSS_SELECTOR, '#_pcmap_list_scroll_container')
while True:
    # 현재 스크롤 위치
    last_height = driver.execute_script("return arguments[0].scrollTop;", container)
    # 컨테이너 아래로 스크롤
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", container)
    # 페이지 로딩 대기
    time.sleep(2)
    # 새로운 스크롤 위치
    new_height = driver.execute_script("return arguments[0].scrollTop;", container)
    # 스크롤이 더 이상 되지 않으면 반복 종료
    if new_height == last_height:
        break
        

# 스크롤 완료 후 검색 결과(식당) 가져오기
restaurant=[] #식당이름 저장 리스트 
for i in range(8, 21):
    try:
        element = driver.find_element(By.XPATH, f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{i}]/div[1]/a[1]/div/div/span[1]')
        text = element.text
        restaurant.append(text)
    except:
        print(f"No element found for index {i}")
        
    
#식당마다 리뷰 가져오기 
rev=[] #전체 식당 리뷰 저장 리스트
for i in range(8, 21):
    res_rev=[] #각각의 식당마다 리뷰 저장 리스트
    
    #첫번째 frame (검색창) 으로 옮김
    driver.switch_to.default_content()
    new_frame_xpath = '//*[@id="searchIframe"]'
    frame = driver.find_element(By.XPATH, new_frame_xpath)
    driver.switch_to.frame(frame)
    
    #식당 클릭, 해당 식당 frame으로 옮겨가기
    element_css_selector = "#_pcmap_list_scroll_container > ul > li:nth-child("+str(i)+") > div.CHC5F > a.tzwk0"
    element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
    element.click()
    time.sleep(2)
        
    #두번째 frame (해당 식당 frame)으로 옮김
    driver.switch_to.default_content()
    new_frame_xpath = '//*[@id="entryIframe"]'
    frame = driver.find_element(By.XPATH, new_frame_xpath)
    driver.switch_to.frame(frame)
    time.sleep(2)
        
    #리뷰 탭 클릭
    review_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "/restaurant/") and contains(@href, "/review?")]'))) #리뷰 위치 영향 안 받게 설정
    review_link.click()
    time.sleep(3)
        
    while True:
        try:
            # "더보기" 버튼을 찾고 클릭
            more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            more_button.click()
            # 리뷰 로드 대기 (10초 동안 기다림)
            time.sleep(5)
            # 현재까지 수집한 리뷰 개수 확인
            current_review_count = len(driver.find_elements(By.XPATH, '//ul[@class="eCPGL"]/li[@class="YeINN"]'))
            # 수집한 리뷰가 100개 이상이라면 반복 종료
            if current_review_count >= 100:
                break
        except Exception as e:
            # 더이상 "더보기" 버튼이 없거나 예외가 발생하면 반복 종료
            print("No more reviews or an error occurred.")
            break
        
        
    for i in range(1, 101): # 더보기 누르지 않은 상태로 최대 10개
        #두번째 frame으로 옮김
        driver.switch_to.default_content()
        new_frame_xpath = '//*[@id="entryIframe"]'
        frame = driver.find_element(By.XPATH, new_frame_xpath)
        driver.switch_to.frame(frame)
        time.sleep(4)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤 끝까지 내리기
        try:
            # 사진 없는 후기는 div.ZZ4OK.IwhtZ 클래스에 위치
            wait = WebDriverWait(driver, 5)
            try: #펼치기가 필요한 경우에는 텍스트를 눌러서 펼치기
                element_css_selector = '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child('+str(i)+') > div.ZZ4OK.IwhtZ > a > span.zPfVt'
                element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
                driver.execute_script("arguments[0].click();", element)
                time.sleep(2)
            except:pass
            #펼쳐진 리뷰들을 크롤링
            review_elements = wait.until(
                    EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="eCPGL"]/li[@class="YeINN"]')))
            review_text = review_elements[i-1].find_element(By.XPATH, './/span[@class="ZZ40K IwhtZ"]').text
            res_rev.append(review_text)
                
        except :
            try:
                # 사진 있는 후기는 div.ZZ4OK 클래스에 텍스트가 위치
                wait = WebDriverWait(driver, 5)
                try: #펼치기가 필요한 경우에는 텍스트를 눌러서 펼치기
                    element_css_selector = '#app-root > div > div > div > div:nth-child(6) > div:nth-child(3) > div.place_section.k5tcc > div.place_section_content > ul > li:nth-child('+str(i)+') > div.ZZ4OK > a > span.zPfVt'
                    element = driver.find_element(By.CSS_SELECTOR, element_css_selector)
                    driver.execute_script("arguments[0].click();", element)
                    time.sleep(2)
                except:pass
                 #펼쳐진 리뷰들을 크롤링
                review_elements = wait.until(
                    EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="eCPGL"]/li[@class="YeINN"]')))
                review_text = review_elements[i-1].find_element(By.XPATH, './/span[@class="zPfVt"]').text
                res_rev.append(review_text)
                    
            except NoSuchElementException : #리뷰 텍스트가 없고 사진만 있는 경우, 결측치 처리
                res_rev.append('NAN')
                
    rev.append(res_rev) #각 식당의 리뷰를 전체 리뷰 리스트에 하나씩 넣기 




In [24]:
#식당명 리스트- 1페이지만 완료해서 50개임
restaurant

['바당한그릇',
 '심바카레',
 '꽁순이네',
 '투썸플레이스곽지과물해변점',
 '하르방짬뽕',
 '문동일셰프의 녹차고을',
 '시루애월',
 '제주돈아 본점',
 '제레미',
 '제주할망밥상 애월점',
 '꽃밥',
 '보말보말',
 '망고레이 본점']

In [25]:
#전체 리뷰 리스트-우선 시간 상 1개식당 * 각 100개 리뷰 (더보기 안 누름) 만 해본 결과 
rev

[['제주도 와서 먹은 식당 중 최고였습니다..!! 물회랑 회덮밥 지인짜 맛있고 신선해욥 황게탕도 게도 여럿이 나눠먹을 정도로 크고 많아서 좋았어용 짱맛\U0001f979\U0001f979 예약하고 가면 사장님께서 서비스도 챙겨주셔서 좋았어요 음식 맛도 좋았지만 사장님과 직원분이 넘 친절하셔서 먹는 내내 기분이 좋았어요 제주에서 마지막 저녁이었는데 최고의 선택이었던 거 같아용 \U0001faf6',
  '제주도 오자마자 물회가 먹고싶어서 찾아보고 왔는데\n너무 맛있게 잘 먹었습니다 \n제 입맛엔 초가 강하지 않아서 좋았고\n서비스로 주신 초밥들도 넘 맛있어서 다 비웠네요\n귤도 감사합니다\n새해 복 많이 받으세요',
  '제주도에서 2박3일돋안 머물면서 먹은 것 중 젤 맛있엇숨도ㅓ!! \n튀김은 선택아니구 필수로 시켜드시요!!\n세상 바삭해서 너무너무 맛있어요! \nㅠㅠ 양도 많고 최고!\n사장님이 넘 친절하시고 스윗하셔서 감동💜\n뷰도 오션뷰니깐 여기서 드시는 거 강추!!! \n다른메뉴 먹으러 또 올게욥!!!💛🙂',
  '제주집 오면 꼭 가는곳 바당한그릇 사장님 친절하시고 음식도 깔끔하게 맛나고 오늘은 대방어 시즌이라 물회에 얹져 나왔는데 김에다 대방어한점 고추장아찌, 파김치 같이 싸서 먹으니 꿀맛이네요(^○^) \n해물볶음소면도 칼칼하니 입에 쫙~ㅎㅎ \n새우튀김도 갓나와서 고소바삭 \n후식서비스로 딸기탕후르 달콤함이 한입가득 즐거운 저녁식사였습니다\n잘먹었습니다 ~~^^',
  '사장님께서 정말 친절하세요^^\n날씨가 너무 춥지만 특물회를 잊지못해서 물회를 시켰어요 제철 방어도 듬뿍 올려주셨구요 먹는 방법도 알려주셨어요 처음 먹어보는 방어도 쫄깃 쫄깃 맛있었구요 서비스도 감사합니다🤗제주오면 또 들릴게요💞 귤도 서비스로 주셔서 잘 먹었어요',
  '요기 식당은 4년전부터 다닌 식당이에요~~ 조끔씩 메뉴는 변하지만 모든 메뉴음식이 현지 제주 해믈을 사용해서 신선하고 맛나요. 특히 여름엔 물회가 짱이죠!!\n오늘 먹은 해물볶음은 정말 순삭할 정도로 밥과 국수 

In [26]:
res_8_20 = rev
res_8_20

[['제주도 와서 먹은 식당 중 최고였습니다..!! 물회랑 회덮밥 지인짜 맛있고 신선해욥 황게탕도 게도 여럿이 나눠먹을 정도로 크고 많아서 좋았어용 짱맛\U0001f979\U0001f979 예약하고 가면 사장님께서 서비스도 챙겨주셔서 좋았어요 음식 맛도 좋았지만 사장님과 직원분이 넘 친절하셔서 먹는 내내 기분이 좋았어요 제주에서 마지막 저녁이었는데 최고의 선택이었던 거 같아용 \U0001faf6',
  '제주도 오자마자 물회가 먹고싶어서 찾아보고 왔는데\n너무 맛있게 잘 먹었습니다 \n제 입맛엔 초가 강하지 않아서 좋았고\n서비스로 주신 초밥들도 넘 맛있어서 다 비웠네요\n귤도 감사합니다\n새해 복 많이 받으세요',
  '제주도에서 2박3일돋안 머물면서 먹은 것 중 젤 맛있엇숨도ㅓ!! \n튀김은 선택아니구 필수로 시켜드시요!!\n세상 바삭해서 너무너무 맛있어요! \nㅠㅠ 양도 많고 최고!\n사장님이 넘 친절하시고 스윗하셔서 감동💜\n뷰도 오션뷰니깐 여기서 드시는 거 강추!!! \n다른메뉴 먹으러 또 올게욥!!!💛🙂',
  '제주집 오면 꼭 가는곳 바당한그릇 사장님 친절하시고 음식도 깔끔하게 맛나고 오늘은 대방어 시즌이라 물회에 얹져 나왔는데 김에다 대방어한점 고추장아찌, 파김치 같이 싸서 먹으니 꿀맛이네요(^○^) \n해물볶음소면도 칼칼하니 입에 쫙~ㅎㅎ \n새우튀김도 갓나와서 고소바삭 \n후식서비스로 딸기탕후르 달콤함이 한입가득 즐거운 저녁식사였습니다\n잘먹었습니다 ~~^^',
  '사장님께서 정말 친절하세요^^\n날씨가 너무 춥지만 특물회를 잊지못해서 물회를 시켰어요 제철 방어도 듬뿍 올려주셨구요 먹는 방법도 알려주셨어요 처음 먹어보는 방어도 쫄깃 쫄깃 맛있었구요 서비스도 감사합니다🤗제주오면 또 들릴게요💞 귤도 서비스로 주셔서 잘 먹었어요',
  '요기 식당은 4년전부터 다닌 식당이에요~~ 조끔씩 메뉴는 변하지만 모든 메뉴음식이 현지 제주 해믈을 사용해서 신선하고 맛나요. 특히 여름엔 물회가 짱이죠!!\n오늘 먹은 해물볶음은 정말 순삭할 정도로 밥과 국수 

In [27]:
import pandas as pd
df_8_20 = pd.DataFrame(res_8_20)
df_8_20

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,제주도 와서 먹은 식당 중 최고였습니다..!! 물회랑 회덮밥 지인짜 맛있고 신선해욥...,제주도 오자마자 물회가 먹고싶어서 찾아보고 왔는데\n너무 맛있게 잘 먹었습니다 \n...,제주도에서 2박3일돋안 머물면서 먹은 것 중 젤 맛있엇숨도ㅓ!! \n튀김은 선택아니...,제주집 오면 꼭 가는곳 바당한그릇 사장님 친절하시고 음식도 깔끔하게 맛나고 오늘은 ...,사장님께서 정말 친절하세요^^\n날씨가 너무 춥지만 특물회를 잊지못해서 물회를 시켰...,요기 식당은 4년전부터 다닌 식당이에요~~ 조끔씩 메뉴는 변하지만 모든 메뉴음식이 ...,음식도 맛있고 뷰도 좋았습니다. 물회와 초밥이 너무 맛있었습니다. 그 동안 많은 음...,사장님도 친절하시고 음식도 너무 맛있었어요!!\n서비스로 주신 후식탕후루까지 너무 ...,제주 겨울여행 마지막날 아침겸 점심으로 찾은 바당한그릇에서 추억앨범 한장 만들어 왔...,애월해안도로 바로 옆 촤고 바다 뷰 맛집! 문어와 전복 가득 해물볶음과 생새우 회 ...,...,제주도올때마다 들리는맛집이에요!!! 해산물 다 신선하고 비린내가 하나도 안나요!! ...,아름다운 바다를 보며 맛있는 물회를 먹을 수 있어요.,냠냠 맛있게 먹다가 중간에 찍었네요ㅠ 새콤달콤물회 싱싱했어요 전복볶음밥도 아주 맛있...,제주의 바다가 바로 옆에 있는 감성 식당입니다. 아침부터 먹는 신선한 물회와 회덮밥...,물회를 진짜 오랜만에 먹는데 새콤매콤 완전 제 스타일 ㅠㅠ 회들도 너무 신선하고 맛...,물회포장해서 맛있게먹은 경험이있어서 재방문했어요 성인두명이서 세트메뉴시켰는데 양이너...,가성비 안 좋음. 기겁. ..물회 밥을 뜨거운것으로 줌 ㅋ,국물이 깔끔하고 맛잇어요,집에서 배달로도 맛있게 먹었지만 예약해서 가봤어요\n친절하시고 연어초밥 서비스까지 ...,너무 친절하신 사장님께서 새우튀김 서비스도 주시고 맛있게 잘 먹고 갑니다 ㅠㅠㅠㅠ ...
1,3시까지만 영업해서 2시반까지는 가야해요! 전 간당간당하게 세이브해서 먹을수 있었습...,최애 카레 최애 돈카츠 \n앉아만 있어도 기분 좋아지는 가게\n\n곽지에 있다가 애...,"갑자기 카레가 땡겨서 찾은 곳인다 만족해요~ \n사장님도 친절하시고, 음식도 맛있고...",지인추천 맛집\n오~~맛있어요\n특히 계란카레? 특이한 조합인데 너무 부드럽고 맛있...,"카레자체가 맛있어서 다 맛있게 잘 먹었어요~!👍👍👍\n매장도 깔끔하고, 분위기도 좋...",근처 펜션 사장님이 추천해주셔서 저녁먹으러갔는데 거기서 일하시는 여자분 넘 친절해서...,원지의하루❣️\n원지님 제주편 보고 방문하게 되었어요. \n요즘 원지님 유툽영상에 ...,지난 여행때 문닫는 날에 딱 걸려 못 들렸던 아쉬움을 지우고자 이번 제주 여행때 방...,맛있게 잘 먹었습니다!!,귀여운 심바도 만나고 맛있는 카레도 먹을 수 있어서 좋았어요. 밥과 카레도 리필 해...,...,친절하고 깔끔하고 맛있어요.,"경치, 분위기 좋고 맛도 좋아요~",굿,계란 카레 엄청 특이하고 맛있네요. 카레 우동 별로에요. 후기 보고 갔는데 면순이 ...,밥 카레의 조화가 맛있어요 ㅎㅎ,계속 생각나서 제주 갈 때마다 들러요 ㅎㅎ,맛집이다. \n강아지를 키우는건지 모르겠지만 강아지는 없었다\n좀더 깔끔한 느낌이었...,카레 양이 조금 적었어요. 인테리어는 예뻐요,카레와 돈까스를 맛있게 잘 먹었습니다,"카레가 안달구 계속 손이가는 맛이에요~ 돈까스두 부드러워서 살살 녹네여 \n다만, ..."
2,완전 맛집!! 제주도가면 꼭 들르는곳 입니다!!,친절하기라면 둘째가라면 서러울 애월 맛집. 양이 정말 많고 재료 신선하고 맛있어요!,진한 몸국에 푸짐한 고기 국수 주민이 찾는 맛집입니다. 특히나 몸국의 진한 맛은 옛...,찐 로컬맛집입니다\n직원모두 친절하시고\n음식들이 신선하고 건강해요♡\n아이와갔는데...,꽁순이네 이번 여행 대박 맛집\n원래 가려던곳은 아니었으나 \n우연치않게 맛본 이집...,제주도에서 몸국 처음먹어봤는데\n속이 뜨뜻해지고 든든해지는 한 끼였습니다.\n서울에...,재주도민 맛집 추천으로 갔는데 육수도 진하고 고기도 두툼하니 맛있었어요!,숙소 주변 고기국수집 검색해서 방문해 봤는데 모든 음식 깔끔하고 맛있었어요!... ...,성인남자들이 배불러서 다 못 먹을 정도로 양 진짜 많아요 ㅋㅋㅋㅋ 저녁까지 배불렀어...,작년에 우연히 지나가다 먹었는데 맛있어서\n이번에 다시 왔어용~~\n지난번엔 국수만...,...,시원하고 맛있어요,몸국도 고기국수도 맛있게 먹었어요~,굿,국수 국물이 조금 짜게 느껴졌는데\n물리지않고 다 싹쓸이..ㅋㅋ\n직원분들이 너무 ...,여기 메뉴 다 맛있어요!! 몸국이랑 고기 국수는 무조건 시켜드세요!! 담에 제주도 ...,"......,................",굿,고기양이 엄청많고 고기 자체가 커요\n김치도 진짜 맛있네용,친절하셔서 좋아요!\n점심시간엔 대기가 있지만 국수라 금방 빠지는듯\n\n의의로 고...,오후에 가면 돔베고기는 품절됩니다.
3,"곽지해수욕장 뷰가 좋고, 투썸이 있어서 좋으네요. 눈보라 쳐서 이동하기 부담되서 갔...",곽지해수옥장앞에 있어서 추운날씨에 몸녹이기 딱인데요 ㅋㅋㅋ,마이투썸하트 네가지맛 다 너무맛있었어요👍👍👍,투썸이니까 맛은 보장 뷰가 좋아요.,투썸케이크는\n언제나 맛있당~~^^\n특히 딸기올라간 케이크는 무조건 맛있음!,무난하게 있을수 있는곳.,곽지바다앞에서 디저트가\n참 맛있어요😊\n디저트는 역시 투썸입니다😍,곽지해수욕장 뷰!,워치 좋고요~뷰도 좋아요~아는 맛🤙,6잔시키고 휴지 욪덩했는데 3장만주시는...,...,해변 왔다가 우연히 들렀는데\n뷰가 너무 좋네요,뷰는 좋아요 :),해변 바로 앞이라 쉬어가기 좋아요,늦어서 포장해서 먹었어요!! 맛있습니다ㅎㅎ,곽지 왔다가 커피 마시러 왔어요\n바다뷰 끝나네요,좋아요,곽지해수욕장에서 시원하게 커피를 즐길 수 있어서 좋았습니다,뷰좋아요,좋아여,오후3시에 갔는데 아직 조각케익이 남아있습니다. 쉬어가기 좋아요.
4,"오랜만에 방문했어요 🥰\n오늘은 기본짜장 7,000원 갈비짬뽕 12,000원 미니탕...",최애 짬뽕집..🫶 \n야들야들한 갈비가 그리워 함박눈을 뚫고 재방문!\n해장을 위해...,갈비짬뽕 대박추천! 갈비완전 부드럽고 불맛제대로에요! 국물도 진하니 조앗씀다ㅋㅋㅋㅋ...,말해 뭐해요~\n오랜 단골입니다 오늘도 겁나게 맛나서 행벅~,맛있는집이라 갈때마다 싹싹 비우고 오는 집입니다 그러나 오늘은 탕수육이 남아 포장 ...,항상 맛있게 먹어요,갈비짬뽕 진짜 맛있어오 입에서 살살 녹아오 고기가,갈비짬뽕 해장 원탑..👍\n남편은 갈비냉우동이 최애 메뉴입니다~~\n이집 갈비가 부...,갈짬….최고에요… 살도 부들부들해서 너무 맛있어요,감사합니다 ㅎㅎ 쿨거래 가능하실까요 나 지금 집 가는 중 가장 권위 있는 국제 인증...,...,맛있고 친절해요,두번째방문인데 일하시는 분들이 너무 진짜 엄청 친절하세요 지금껏 갔던 식당중에서 제...,"직장인 점심 핫플레이스입니다 ㅋㅋㅋㅋㅋㅋ\n갈비짬뽕 진짜 맛있는데 웨이팅 주의,,,...",점심시간 피해서가니까 한산했어요.\n딱예상했던맛이에요.\n특별히맛있진안았어요,제주여행중 입이즐거웠던곳 너무 잘먹었어요\n담에 제주여행때 꼭디시갈곳,기본 짬뽕하고 갈비짬뽕 탕수육 맛있어요!\n들깨뼈다귀짬뽕도 있는데 이건 감자탕맛나는...,짬뽕은 하르방이죠 맛있어요,제주여행와서 마지막 식사로 선택한 갈비짬뽕!!\n제주에서 먹음 음식중에 가격대비 만...,맛도 좋고 주변지나가면 항상 가는곳인데\n무엇보다 기분좋았던건 직원분들 다들 친절하...,맛잇어요
5,식감 좋은 녹차면에 들깨국물 넘 고소하고\n맛있어요. 친절하시고 음식들이 다 맛있어...,"제주여행 2일차 첫끼, 늦은 아침으로 먹은 고사리육개장에 전복세트도 맛있고 녹차칼국...",음식이 정말 맛있네요\n여행중 간단하게 한그릇하기 딱입니다.\n문동일쉐프님 얼굴도 ...,1년만에 다시방문한 문동일셰프의녹차고을 #고사리육개장 맛있다~ 친구는 칼국수와떡갈비...,24년 새해에 첫번째 방문해서 몸보신용\n전복찜과 소안심스테이크 고사리육개장밥 가끔...,간만에 왔네요\n녹차칼국수 항상 맛있어

In [28]:
df_8_20 =   df_8_20.transpose()

In [29]:
df_8_20.to_csv('rev_8_20.csv',encoding='utf-8-sig')